In [1]:
from src.robot_manager import RobotController
from src.physics_utils import PyBulletSim, CameraManager
import pybullet as p
import pybullet_data
import numpy as np
import time

sim = PyBulletSim(p.GUI)
cam = CameraManager(target_pos=[0, 0, 0], distance=1.2, yaw=90, pitch=-45)
robot = RobotController("franka_panda/panda.urdf")

robot.ee_velocity = 0.2
robot.gripper_rot_velocity = 2

time.sleep(1)
while p.isConnected():
	keys = p.getKeyboardEvents()

	# Default velocities
	lin_vel = [0, 0, 0]
	rot_vel = 0

	# --- Check for key presses to set velocities ---
	# Gripper rotation (continuous while holding key)
	if ord('q') in keys and keys[ord('q')] & p.KEY_IS_DOWN:
		rot_vel = robot.gripper_rot_velocity
	if ord('e') in keys and keys[ord('e')] & p.KEY_IS_DOWN:
		rot_vel = -robot.gripper_rot_velocity

	# XY Movement (continuous while holding key)
	if p.B3G_RIGHT_ARROW in keys and keys[p.B3G_RIGHT_ARROW] & p.KEY_IS_DOWN:
		lin_vel[0] = robot.ee_velocity # +X
	if p.B3G_LEFT_ARROW in keys and keys[p.B3G_LEFT_ARROW] & p.KEY_IS_DOWN:
		lin_vel[0] = -robot.ee_velocity # -X
	if p.B3G_UP_ARROW in keys and keys[p.B3G_UP_ARROW] & p.KEY_IS_DOWN:
		lin_vel[1] = robot.ee_velocity # +Y
	if p.B3G_DOWN_ARROW in keys and keys[p.B3G_DOWN_ARROW] & p.KEY_IS_DOWN:
		lin_vel[1] = -robot.ee_velocity # -Y

	# --- Apply velocities to the robot ---
	robot.move_ee_velocity(lin_vel)
	robot.rotate_gripper_velocity(rot_vel)

	# Keep the simulation running
	p.stepSimulation()
	time.sleep(sim.time_step)

sim.close()